In [ ]:
'''Exploratory Data Analysis and their Visualisations
Investigating Questions
1. Which locations do the crimes mostly occur ?'''

plt.figure(figsize = (15, 10))
sns.countplot(y= 'LocationDescription', data = Crimes_copy, order = Crimes_copy['LocationDescription'].value_counts().iloc[:10].index)

In [ ]:
'''
 Which crimes happen the most ?
'''
plt.figure(figsize = (15, 10))
sns.countplot(y= 'PrimaryType', data = Crimes_copy, order = Crimes_copy['PrimaryType'].value_counts().iloc[:10].index)

In [ ]:
'''What is the trend of the crimes happening along the years?'''
crime_data_for_trend = Crimes_copy.pivot_table('ID', aggfunc=np.size, columns='PrimaryType', index=Crimes_copy.index.date, fill_value=0)
crime_data_for_trend.index = pd.DatetimeIndex(crime_data_for_trend.index)
plot = crime_data_for_trend.rolling(365).sum().plot(figsize=(12, 24), subplots=True, layout=(-1, 3), sharex=False, sharey=False)

In [ ]:
'''The pattern of crimes
1.Trend of Crimes occurring over the years'''


plt.figure(figsize=(13,8))
Crimes_copy.resample('M').size().plot(legend=False)
plt.title('Trend of Crimes occurring over the years')
plt.xlabel('Months')
plt.ylabel('Number of crimes')
plt.show()

In [ ]:
'''2. Crimes happening monthwise'''
Crimes_copy.groupby([Crimes_copy.index.month]).size().plot(kind='barh')
plt.ylabel('Months of the year')
plt.xlabel('Number of crimes')
plt.title('Number of crimes by month of the year')
plt.show()

In [ ]:
days = ['Monday','Tuesday','Wednesday',  'Thursday', 'Friday', 'Saturday', 'Sunday']
Crimes_copy.groupby([Crimes_copy.index.dayofweek]).size().plot(kind='barh')
plt.ylabel('Days of the week')
plt.yticks(np.arange(7), days)
plt.xlabel('Number of crimes')
plt.title('Number of crimes by day of the week')
plt.show()

In [ ]:
plt.figure(figsize=(11,4))
Crimes_copy.resample('D').size().rolling(365).sum().plot()
plt.title('Rolling sum of all crimes from 2001 - 2020')
plt.ylabel('Number of crimes')
plt.xlabel('Days')
plt.show()

In [ ]:
Crimes = Crimes.loc[(Crimes['X Coordinate']!=0)]
sns.lmplot('X Coordinate', 
           'Y Coordinate',
           data=Crimes[:],
           fit_reg=False, 
           hue="District",
           palette='Dark2',
           size=14,
           ci=2,
           scatter_kws={"marker": "D", 
                        "s": 10})
ax = plt.gca()
ax.set_title("All Crime Distribution per District")
plt.savefig("incorrectdistrict.png")

In [ ]:
col2 = ['Date','Primary Type','Arrest','Domestic','District','X Coordinate','Y Coordinate']
multiple_crimes = Crimes[col2]
multiple_crimes = multiple_crimes[multiple_crimes['Primary Type']\
                  .isin(['HOMICIDE','CONCEALED CARRY LICENSE VIOLATION','NARCOTICS','WEAPONS VIOLATION'])]
# clean some rouge (0,0) coordinates
multiple_crimes = multiple_crimes[multiple_crimes['X Coordinate']!=0]
multiple_crimes.head()
g = sns.lmplot(x="X Coordinate",
               y="Y Coordinate",
               col="Primary Type",
               data=multiple_crimes.dropna(), 
               col_wrap=2, size=6, fit_reg=False, 
               sharey=False,
               scatter_kws={"marker": "D",
                            "s": 10})

In [ ]:
df_public_peace =  Crimes[Crimes['Primary Type'] == 'PUBLIC PEACE VIOLATION']
df_public_data = pd.DataFrame({"Counts": df_public_peace['Description'].value_counts(), "Description" : df_public_peace['Description'].value_counts().index})
df_public_data.reset_index(inplace=True)
df_public_data = df_public_data.drop(columns=['index'], axis = 1)
df_public_data.head()
plt.figure(figsize = (15, 7))
sns.barplot(y ="Description", x = "Counts", data = df_public_data, palette="cool")